# Instrumentify data pre processing: logmel

This notebook process the data OpenMIC to create a pipeline and transform the audio streams into audio-visual representations such as Log-Mel and Gammatone spectrograms.

In [ ]:
# Common commands
!zip -r '/content/folder/"NewFileName.zip"' '/content/folder/"OrginalFileName"'
!unzip <filepath>
!wget <url>
%cp <zip_to_copy> /content/drive/MyDrive/ESP3201/Datasets/<dataset name>
plt.imsave() #for saving only the image without axes/borders and all
!du -h -s <filepath> #check size

## How processing works
transform (short time fft/ fourier/ constant q) --> spectogram (mel/gamma) --> log scale

# OpenMIC

In [59]:
!pwd

/home/studio-lab-user/sagemaker-studiolab-notebooks/ESP3201-Instrument-indentification


In [ ]:
# Download original zip for OpenMIC
!wget https://zenodo.org/record/1432913/files/openmic-2018-v1.0.0.tgz

In [16]:
# Extract and delete the zip
!tar --extract --file openmic-2018-v1.0.0.tgz
!rm openmic-2018-v1.0.0.tgz

In [ ]:
# HELPFUL for getting list of audio files for running mel/gammatone on all, also to verify
import os
filelist=[]
for root, dirs, files in os.walk("openmic-2018/audio"):
	for file in files:
        #append the file name to the list
		filelist.append(os.path.join(root,file))
print(len(filelist))

In [14]:
# to remove the folder if needed
!rm -r openmic-2018

# Convert to log mel


### Import libraries

In [3]:
import shutil
import os

import librosa
import librosa.display
import IPython.display as ipd
import matplotlib
import numpy as np

### Rename folder to ogg audio files

In [2]:
# rename original audio to audio-ogg for clarity
os.rename("openmic-2018/audio","openmic-2018/audio-ogg")

In [19]:
# HELPFUL for getting list of audio files for running mel/gammatone on all
# use this here to rename audio to audio-ogg and get new file list
import os
oggfilelist=[]
for root, dirs, files in os.walk("openmic-2018/audio-ogg"):
	for file in files:
        #append the file name to the list
		oggfilelist.append(os.path.join(root,file))
print (len(oggfilelist))

20000


In [5]:
# check size for no specific reason
!du -h -s openmic-2018
# !rm -r openmic-2018/audio-wav/

2.9G	openmic-2018


In [22]:
# check size for no specific reason
!du -h -s openmic-2018

5.2G	openmic-2018


### Create conversion function for one file

In [2]:
# parameters of the librosa.feature.melspectrogram is added for flexibility
def render_logmel_from_file(path, fft_window, hop_size, mel_bands, save_path):
    # load the file using Librosa and obtain the scale and sampling rate.
    scale, sampling_rate = librosa.load(path)
    # generate the mel_spectrogram and convert to dB (logmel)
    mel_spectrogram = librosa.feature.melspectrogram(scale, sr=sampling_rate, n_fft=fft_window, hop_length=hop_size, n_mels=mel_bands)
    log_mel_spectrogram = librosa.power_to_db(mel_spectrogram)
    
    # convert spectrogram numpy to image
    img = librosa.display.specshow(log_mel_spectrogram, x_axis="time", y_axis="mel", sr=sampling_rate)
    
    # save the figure as a png
    matplotlib.pyplot.axis("off")
    matplotlib.pyplot.savefig(save_path, bbox_inches='tight',pad_inches = 0)
    # print(save_path)
    matplotlib.pyplot.close()
    
matplotlib.rcParams.update({'figure.max_open_warning': 0})

In [28]:
# test on one file if needed
# ogg_file = "/home/studio-lab-user/sagemaker-studiolab-notebooks/openmic-2018/audio-ogg/000/000046_3840.ogg"
# logmelfile = os.path.splitext(ogg_file)[0]+'.png'
# logmelfile = logmelfile.replace(logmelfile[logmelfile.index("audio-ogg"):],"audio-logmel" + logmelfile[logmelfile.index("audio-ogg")+9:])
# render_logmel_from_file(ogg_file, 2048, 512, 128, logmelfile)

### Bulk convert


In [15]:
# defining the function to ignore the files if present in any folder
def ignore_files(dir, files):
    return [f for f in files if os.path.isfile(os.path.join(dir, f))]

# remove the file used in testing
!rm -r openmic-2018/audio-logmel

# create data folder structure for logmel of similar structure to audio-ogg
shutil.copytree('openmic-2018/audio-ogg/',
                'openmic-2018/audio-logmel/',
                ignore=ignore_files)

'openmic-2018/audio-logmel/'

In [11]:
ogg_file = "openmic-2018/audio-ogg/000/000046_3840.ogg"

In [32]:
def bulk_convert(ogg_file):
    logmelfile = os.path.splitext(ogg_file)[0]+'.png'
    logmelfile = logmelfile.replace(logmelfile[logmelfile.index("audio-ogg"):],"audio-logmel" + logmelfile[logmelfile.index("audio-ogg")+9:])
    render_logmel_from_file(ogg_file, 2048, 512, 128, logmelfile)
    # print(logmelfile)
    
#bulk_convert(ogg_file)

In [ ]:
#!pip install tqdm
from tqdm import tqdm
from multiprocessing import Pool

# convert with multiprocessing, 2 coz with 3 for some reason the third one stops working v early
p = Pool(processes=2)
a = list(tqdm(p.imap(bulk_convert, oggfilelist), total = 20000))
# p.map(bulk_convert, oggfilelist)
# the tqdm not rly working sadly :(
# IF IT GETS STUCK, RESTART KERNEL 
# for ogg in tqdm(oggfilelist):
#     bulk_convert(ogg)

In [26]:
import os
logmelfilelist=[]
for root, dirs, files in os.walk("openmic-2018/audio-logmel"):
	for file in files:
        
        #append the file name to the list
		logmelfilelist.append(os.path.join(root,file))
print(len(logmelfilelist))

19990


# Troubleshooting
for any data that is unsuccessfully converted

In [12]:
%cd ..

/home/studio-lab-user/sagemaker-studiolab-notebooks


In [ ]:
import os
oggfilelist=[]
for root, dirs, files in os.walk("openmic-2018/audio-ogg"):
    for file in files:
        #append the file name to the list
        oggfilelist.append(os.path.splitext(file)[0])
        #oggfilelist.append(file)
print(len(oggfilelist))
print(oggfilelist[1])

In [ ]:
import os
logmelfilelist=[]
for root, dirs, files in os.walk("openmic-2018/audio-logmel"):
	for file in files:
        
        #append the file name to the list
		logmelfilelist.append(os.path.splitext(file)[0])
print(len(logmelfilelist))
print(logmelfilelist[1])

In [17]:
diff = list(set(logmelfilelist) ^ set(oggfilelist[:19988])) # Symmetric diff
print(len(diff))
print(diff)

26
['087246_145920', '155311_453120', '007116_107520-checkpoint', '029496_26880', '019964_88320', '155204_7680', '155294_184320', '052862_364800', '138182_311040', '155293_26880', '042332_11520', '155245_629760', '155278_211200', '149448_88320', '155225_126720', '007122_218880-checkpoint', '155307_211200', '115592_180480', '155197_34560', '155295_76800', '155310_372480', '104759_314880', '126419_207360', '155233_364800', '073544_533760', '062742_15360']


In [18]:
diff = list(set(logmelfilelist) ^ set(oggfilelist)) # check diff
print(len(diff))
print(diff)

14
['087246_145920', '007116_107520-checkpoint', '029496_26880', '019964_88320', '052862_364800', '138182_311040', '042332_11520', '149448_88320', '007122_218880-checkpoint', '115592_180480', '104759_314880', '126419_207360', '073544_533760', '062742_15360']


In [ ]:
diff = []
for file in oggfilelist:
    if file not in logmelfilelist:
        diff.append(file)
print(len(diff))
print(diff)

In [44]:
file_paths = []
for file in diff:
    path = "openmic-2018/audio-ogg/"
    path += file[:3] + "/" + file + ".ogg"
    file_paths.append(path)

print(len(file_paths))
print(file_paths)

11
['openmic-2018/audio-ogg/029/029496_26880.ogg', 'openmic-2018/audio-ogg/042/042332_11520.ogg', 'openmic-2018/audio-ogg/052/052862_364800.ogg', 'openmic-2018/audio-ogg/062/062742_15360.ogg', 'openmic-2018/audio-ogg/073/073544_533760.ogg', 'openmic-2018/audio-ogg/087/087246_145920.ogg', 'openmic-2018/audio-ogg/104/104759_314880.ogg', 'openmic-2018/audio-ogg/115/115592_180480.ogg', 'openmic-2018/audio-ogg/126/126419_207360.ogg', 'openmic-2018/audio-ogg/138/138182_311040.ogg', 'openmic-2018/audio-ogg/149/149448_88320.ogg']


In [ ]:
for path in file_paths:
    bulk_convert(path)
print("done")